In [1]:
from LSTM import LSTMNetworkExplicit
from Transformer import TransformerDecoderWithRoPE
import torch

In [11]:
# network = TransformerDecoderWithRoPE(vocab_size=128, hidden_size=768, num_layers=16)
network = LSTMNetworkExplicit(vocab_size=128, hidden_size=768, num_lstm_layers=16)

In [13]:
# network.load_state_dict(torch.load('transformer/checkpoints/transformer_epoch32_batch7000.pt', weights_only=True, map_location=torch.device('cpu'))['model_state_dict'])
network.load_state_dict(torch.load('lstm/lstm_language_model_epoch2_batch4000.pth', weights_only=True, map_location=torch.device('cpu')))

<All keys matched successfully>

In [14]:
# Test the network with some input
from tokenizer import AsciiTokenizer
tokenizer = AsciiTokenizer()

In [15]:
input_text = "Jesus Chri"
input_ids = tokenizer.encode(input_text)
input_tensor = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension
max_token_length = 32
for _ in range(max_token_length):
    with torch.no_grad():
        output = network(input_tensor)
    next_token_logits = output[0, -1, :]  # Get logits for the last time step
    # sample the next token from random distribution
    next_token_id = torch.multinomial(torch.softmax(next_token_logits, dim=-1), num_samples=1).item()
    input_tensor = torch.cat([input_tensor, torch.tensor([[next_token_id]])], dim=1)
generated_ids = input_tensor.squeeze(0).tolist()
generated_text = tokenizer.decode(generated_ids)
print(generated_text)

Jesus Christ in 1839. He deputed himself f


In [16]:
querries = [
    "2 + 2 =",
    "red, green, bl",
    "The meaning of life is",
    "In the future AI",
    "A cute cat sat on",
    "In the beginning there was",
    "1 2 3",
    "#inclu",
    "II world war was started in",
    "first, seco",
    "Marie Cu",
    "Nobel Price in Physics was awarded to",
    "Cats are",
    "Research is creative",
    "Reality is the sum",
    "Hund ist sehr unruh",
    "Jesus Chri"
]

In [17]:
ans = []
for query in querries:
    input_ids = tokenizer.encode(query)
    input_tensor = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension
    max_token_length = 32
    for _ in range(max_token_length):
        with torch.no_grad():
            output = network(input_tensor)
        next_token_logits = output[0, -1, :]  # Get logits for the last time step
        next_token_id = torch.argmax(next_token_logits, dim=-1).item()
        input_tensor = torch.cat([input_tensor, torch.tensor([[next_token_id]])], dim=1)
    generated_ids = input_tensor.squeeze(0).tolist()
    generated_text = tokenizer.decode(generated_ids)
    ans.append((query, generated_text))

In [18]:
ans

[('2 + 2 =', '2 + 2 = 10 \x01 1 + 1 + 1 + 1 + 1 + 1 + 1 '),
 ('red, green, bl', 'red, green, black and white and the sides of t'),
 ('The meaning of life is',
  'The meaning of life is the same as the story of the st'),
 ('In the future AI', 'In the future AI has a significant importance in'),
 ('A cute cat sat on', 'A cute cat sat on the streets of the city of Cata'),
 ('In the beginning there was',
  'In the beginning there was a series of storms of the same '),
 ('1 2 3', '1 2 3 1 1 1 1 September 2019\n\nReferen'),
 ('#inclu', '#includes the following species:\n\n Con'),
 ('II world war was started in',
  'II world war was started in 1945 and the second segment of '),
 ('first, seco', 'first, second and third placed teams were d'),
 ('Marie Cu', 'Marie Curie and the Conservative Party o'),
 ('Nobel Price in Physics was awarded to',
  'Nobel Price in Physics was awarded to the best physical and physical '),
 ('Cats are', 'Cats are a substance that is a product o'),
 ('Research is cre